# Assignment 7

### Part A

In [2]:
import numpy as np
import pandas as pd
import math

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })

portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    base = (P["Amount Invested"]).sum()
    exp = sum((rf + P["Beta"] * mkt_rp) * (P["Amount Invested"]/ base))
    return exp
expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [4]:
def volatility(P, cor):
    # Write your code here
    base = (P["Amount Invested"]).sum()
    var = sum(P["Volatility"] ** 2 * (P["Amount Invested"]/ base) ** 2)
    var = var + 2 * cor * P["Volatility"][0] * (P["Amount Invested"][0]/ base)* P["Volatility"][1]*(P["Amount Invested"][1]/ base)
    sd = math.sqrt(var)
    return sd
volatility(portfolio, -0.2) 

0.24083189157584595

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    weight = vol/ mkt_vol
    ep = rf + mkt_rp * weight 
    return ep


expected_return2(0.02, 0.05, 0.16, 0.2408) 

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [4]:
def calculate_beta(sub_data):
    cov = sub_data["r_excess"].cov(sub_data["mkt_excess"])
    var = sub_data["mkt_excess"].var()
    beta = cov / var
    return pd.Series({"beta": beta})

def CAPM_beta(return_path, factors_path):
    #Write your code here
    rp = pd.read_csv(return_path)
    rp['date'] = rp.apply(lambda row: f"{row['year']}{row['month']:02d}", axis=1)
    rp['date'] = rp['date'].astype(int)
    rp = rp.drop(['year', 'month'], axis=1)
    
    fp = pd.read_csv(factors_path)
    fp["Mkt-RF"] = fp["Mkt-RF"]/100
    fp["RF"] = fp["RF"] / 100
   
    merged_df = pd.merge(rp, fp, on = 'date', how = 'left')

    merged_df["r_excess"] = merged_df["RET"] - merged_df["RF"]
    merged_df["mkt_excess"] = merged_df["Mkt-RF"]
    beta_data = merged_df.groupby("CUSIP").apply(calculate_beta)
    merged_df = pd.merge(merged_df, beta_data, left_on="CUSIP", right_index=True, how="left")
    merged_df["expected_return"] = merged_df["RF"] + merged_df["beta"] * merged_df["mkt_excess"]

    output = pd.DataFrame(data = {
        "CUSIP": merged_df["CUSIP"].unique(),
        "EXP_RETURN": merged_df.groupby("CUSIP")["expected_return"].last().values,
        "BETA": merged_df.groupby("CUSIP")["beta"].last().values,
    })
    
    return output


CAPM_beta("../../datafiles/A7/monthlycrsp_mod7.csv", "../../datafiles/A7/factors.CSV")

CUSIP  EXP_RETURN       BETA
0  36020A   -0.012955  -0.483703
1  36021A   -0.118081  -1.124363
2  36022A    0.140373  -5.149374
3  36023A    0.387152  10.187062

<img src="image-20231114-141218.png" width="" align="" />

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [20]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    coeffs = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    slope = coeffs[0]
    return -0.3066162755163193 * slope
data = CAPM_beta("/work/monthlycrsp_mod7.csv", "/work/F-F_Research_Data_Factors.CSV")
slope(data)

0.007981588120535642

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=60127f27-e0e6-4612-9172-063a2230cfd2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>